<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Google-Cloud-Speech-to-Text-API-Streaming-example" data-toc-modified-id="Google-Cloud-Speech-to-Text-API-Streaming-example-1">Google Cloud Speech-to-Text API Streaming example</a></span></li></ul></div>

# Google Cloud Speech-to-Text API Streaming example

- 1분 이상의 긴 오디오 파일을 Streaming 방식으로 처리하는 샘플
- https://cloud.google.com/speech-to-text/docs/streaming-recognize
- google.cloud.speech.SpeechClient().**streaming_recognize(streaming_config, requests)**
- **Audio file size limit : 10M bytes** => 10M 이상의 파일을 처리하기 위해 wave file을 10M 이하 크기로 잘라서 변환하도록 개선

In [3]:
#!pip install pyaudio

In [4]:
import pyaudio
import wave
import os
import io
import csv

import numpy as np

from nltk.tokenize import sent_tokenize, word_tokenize

# for speaker_diarization (use speech_v1p1beta1)
from google.cloud import speech_v1p1beta1 as speech
from google.cloud.speech_v1p1beta1 import enums
from google.cloud.speech_v1p1beta1 import types

In [5]:
def transcribe_streaming(stream_file):
    """Streams transcription of the given audio file."""
    words_with_tag = []
    transcripts = []

    client = speech.SpeechClient()

    with io.open(stream_file, 'rb') as audio_file:
        content = audio_file.read()

    # In practice, stream should be a generator yielding chunks of audio data.
    stream = [content]

    requests = (types.StreamingRecognizeRequest(audio_content=chunk)
                for chunk in stream)

    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code='ko-KR',
        enable_automatic_punctuation=True,
        enable_word_time_offsets=True,
        enable_speaker_diarization=True,    # 한국어 지원 안됨 (speaker_tag가 모두 동일인으로 분류됨)
        diarization_speaker_count=3)

    streaming_config = types.StreamingRecognitionConfig(config=config)
    
    # streaming_recognize returns a generator.
    responses = client.streaming_recognize(streaming_config, requests)
    
    for response in responses:
        for result in response.results:
            alternatives = result.alternatives
            for alternative in alternatives:
                print(u'Transcript: {}'.format(alternative.transcript))
                transcripts.append(alternative.transcript)    # punctuation 포함된 문장을 사용하기 위해 저장
                for words in alternative.words:
                    word = words.word
                    start_time = round(words.start_time.seconds + words.start_time.nanos * 1e-9, 3)
                    end_time = round(words.end_time.seconds + words.end_time.nanos * 1e-9, 3)
                    speaker_tag = words.speaker_tag
                    words_with_tag.append([word, start_time, end_time, speaker_tag])    # [word, start_time, end_time, speaker_tag]
        print()
    return words_with_tag, transcripts

In [6]:
def write_wave_frames(frames):
        wave_stream = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
        wave_stream.setnchannels(CHANNELS)
        wave_stream.setsampwidth(FORMAT)
        wave_stream.setframerate(RATE)
        wave_stream.writeframes(b''.join(frames))
        wave_stream.close()

In [7]:
# instantiate PyAudio
audio = pyaudio.PyAudio()

In [8]:
# open wave file
FILE_NAME = "19m_20181220_deominju_16000"
wf = wave.open("audio_file/"+ FILE_NAME + ".wav", 'rb')

In [9]:
# initialize variables
CHUNK = 1024   # 1024 bytes of data read from the buffer
FRAME_SIZE = CHUNK * 4
WAVE_OUTPUT_FILENAME = "wave_output.wav"    # temperary wave file for streaming 
FORMAT=audio.get_sample_size(wf.getsampwidth())
CHANNELS=wf.getnchannels()
RATE=wf.getframerate()
words_with_tags = []
transcripts_arr = []
frames = []
f_count = 0

In [10]:
# read data (first frames)
data = wf.readframes(CHUNK)

# transcribe stream  : wave file을 10M 이하로 잘라서, google STT API에 stream 방식으로 요청
while len(data) > 0:
    frames.append(data)
    f_count += 1
    if f_count == FRAME_SIZE:
        # write frames to wave file for frames
        write_wave_frames(frames)
        
        # transcribe by streaming
        words_with_tag, transcripts = transcribe_streaming(WAVE_OUTPUT_FILENAME)    # frames를 직접 전달 시 google에서 grpc.RpcError 발생하면 cancel 됨 
        words_with_tags.append(words_with_tag)
        transcripts_arr.append(transcripts)
        
        # frame initialize
        frames = []
        f_count = 0
    
    # read data (continue)
    data = wf.readframes(CHUNK)

# transcribe for last frames
write_wave_frames(frames)

# last frames
words_with_tag, transcripts = transcribe_streaming(WAVE_OUTPUT_FILENAME)
words_with_tags.append(words_with_tag)
transcripts_arr.append(transcripts)

Transcript: 네 반갑습니다 제 115차 정책조정회의를 원내대표에 모두 바로 시작하겠습니다. 11 12 월 임시 이렇게 분위기가 아 이제 일주일 앞으로 다가왔습니다. 여야가 민생법안 처리를 위해 논의 속도를 더 내야겠습니다. 꼭 처리해야 할 과제는 6300번 산업안전보건법입니다. 6003번 오늘의 연애 법안소위에서 눈이 될 예정입니다. 오늘 합의가 이루어지지 않을 경우 27번의 처리가 사실상 어렵습니다. 자유한국당도 아이들이 하는 마음이 조금이라도 있다면 6003번 노니 접속 구매합니다. 지난번 정기국회 때처럼 법원 처리를.

Transcript: 연 시키려는 무슨 국민이 용납하지 않을 것입니다. 우리 다음은 오늘 처리될 수 있도록 최선을 다해서 열린 자세로 소방서에 임하겠습니다. 그러나 끝내 자유한국당이 반대한다면 패스트트랙을 통해 처리하는 방안을 추진 하겠습니다. 위험의 외주화를 막을 산업안전보건법 또 반드시 처리하겠습니다. 더 이상 내몰리지 않기 위해서 필요한 법안입니다. 다행히 어제 황영희 고용노동청 위해서 27일 본회의에서 산업안전보건법을 처리하기 여야간 의견이 모아졌다습니다. 앞으로 돈이 과정에서 차질 없이 처리될 수 있도록 기도합니다.

Transcript: 협상을 총괄하는 비건 미국 특별대표가 어제 방황하면서 주목할만한 메세지를 잡혀 있습니다. 인도적 차원에서 대북지원을 확실히 보장하고 미국인의 북한 여행 금지조치를 재검토하겠다고 밝혔다입니다. 남궁민 사이를 갈라놓을 어떤지 난 70년에 적대감을 뛰어넘을 수 있도록 북한과 확장하겠다는 운동했습니다. 그동안은 엄격한 대북제재 방침을 고수했던 미국이 유화책을 제시한 것입니다. 미국 정부가 인도적 차원에서 대북지원을 북미대화를 다시 진전 시키겠다는 강한 의지를 밝힌 것으로 해석된다. 북한이 어떤 반응을 보이는 양에 따라 행동할 교착상태를 보이던 대화가 재개 될 가능성도 충분히 있어 보입니다. 오늘과 내일 추석 대표회의 한미워킹그룹 2차 회의를 통해 한미 양국이 국민.

Transcript: 새로

In [11]:
# close PyAudio
audio.terminate()

In [12]:
corpus = ""
for transcripts in transcripts_arr:
    for transcript in transcripts:
        corpus += transcript + " "
# print(corpus)

In [13]:
sentence = sent_tokenize(corpus)    # 문장단위 분리
sentence

['네 반갑습니다 제 115차 정책조정회의를 원내대표에 모두 바로 시작하겠습니다.',
 '11 12 월 임시 이렇게 분위기가 아 이제 일주일 앞으로 다가왔습니다.',
 '여야가 민생법안 처리를 위해 논의 속도를 더 내야겠습니다.',
 '꼭 처리해야 할 과제는 6300번 산업안전보건법입니다.',
 '6003번 오늘의 연애 법안소위에서 눈이 될 예정입니다.',
 '오늘 합의가 이루어지지 않을 경우 27번의 처리가 사실상 어렵습니다.',
 '자유한국당도 아이들이 하는 마음이 조금이라도 있다면 6003번 노니 접속 구매합니다.',
 '지난번 정기국회 때처럼 법원 처리를.',
 '연 시키려는 무슨 국민이 용납하지 않을 것입니다.',
 '우리 다음은 오늘 처리될 수 있도록 최선을 다해서 열린 자세로 소방서에 임하겠습니다.',
 '그러나 끝내 자유한국당이 반대한다면 패스트트랙을 통해 처리하는 방안을 추진 하겠습니다.',
 '위험의 외주화를 막을 산업안전보건법 또 반드시 처리하겠습니다.',
 '더 이상 내몰리지 않기 위해서 필요한 법안입니다.',
 '다행히 어제 황영희 고용노동청 위해서 27일 본회의에서 산업안전보건법을 처리하기 여야간 의견이 모아졌다습니다.',
 '앞으로 돈이 과정에서 차질 없이 처리될 수 있도록 기도합니다.',
 '협상을 총괄하는 비건 미국 특별대표가 어제 방황하면서 주목할만한 메세지를 잡혀 있습니다.',
 '인도적 차원에서 대북지원을 확실히 보장하고 미국인의 북한 여행 금지조치를 재검토하겠다고 밝혔다입니다.',
 '남궁민 사이를 갈라놓을 어떤지 난 70년에 적대감을 뛰어넘을 수 있도록 북한과 확장하겠다는 운동했습니다.',
 '그동안은 엄격한 대북제재 방침을 고수했던 미국이 유화책을 제시한 것입니다.',
 '미국 정부가 인도적 차원에서 대북지원을 북미대화를 다시 진전 시키겠다는 강한 의지를 밝힌 것으로 해석된다.',
 '북한이 어떤 반응을 보이는 양에 따라 행동할 교착상태를 보이던 대화가 재개 될 가능성도 충분히 있어 보입니다.',
 '오늘과 내일 추석 

In [14]:
# 스크립트 만 Text 파일로 저장 (with newline)
text_file = "text/" + FILE_NAME + ".txt"
with open(text_file, 'w', encoding='utf-8') as f:
    for sent in sentence:
        f.write(sent + "\n")

In [15]:
# print(words_with_tags)
words_array = [wt for wts in words_with_tags for wt in wts]
words_array

[['네', 0.4, 6.9, 1],
 ['반갑습니다', 6.9, 7.2, 1],
 ['제', 7.2, 7.8, 1],
 ['115차', 7.8, 8.7, 1],
 ['정책조정회의', 8.7, 9.5, 1],
 ['를', 9.5, 9.9, 1],
 ['원내대표에', 9.9, 11.0, 1],
 ['모두', 11.0, 11.2, 1],
 ['바로', 11.2, 11.5, 1],
 ['시작하겠습니다', 11.5, 12.0, 1],
 ['11', 12.0, 15.5, 1],
 ['12', 15.5, 16.3, 1],
 ['월', 16.3, 16.6, 1],
 ['임시', 16.6, 17.1, 1],
 ['이렇게', 17.1, 17.5, 1],
 ['분위기가', 17.5, 18.2, 1],
 ['아', 18.2, 18.9, 1],
 ['이제', 18.9, 19.4, 1],
 ['일주일', 19.4, 19.7, 1],
 ['앞으로', 19.7, 20.2, 1],
 ['다가왔습니다', 20.2, 20.6, 1],
 ['여야가', 20.6, 22.6, 1],
 ['민생법안', 22.6, 23.6, 1],
 ['처리를', 23.6, 24.1, 1],
 ['위해', 24.1, 24.3, 1],
 ['논의', 24.3, 25.3, 1],
 ['속도를', 25.3, 26.1, 1],
 ['더', 26.1, 26.3, 1],
 ['내야겠습니다', 26.3, 26.8, 1],
 ['꼭', 26.8, 28.6, 1],
 ['처리해야', 28.6, 29.3, 1],
 ['할', 29.3, 29.6, 1],
 ['과제는', 29.6, 30.5, 1],
 ['6300번', 30.5, 31.9, 1],
 ['산업안전보건법', 31.9, 33.6, 1],
 ['입니다', 33.6, 34.0, 1],
 ['6003번', 34.0, 36.0, 1],
 ['오늘', 36.0, 36.3, 1],
 ['의', 36.3, 36.8, 1],
 ['연애', 36.8, 37.0, 1],
 ['법안소위에서', 

In [16]:
# CSV 파일로 저장
csv_file = "text/" + FILE_NAME + ".csv"
with open(csv_file, 'w', encoding='utf-8', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["word", "start_time", "end_time", "speaker_tag"])
    for words in words_array:
        wr.writerow(words)